# Python 3

## Итераторы, генераторы, исключения


---

### 0. "Iterable object"
Питоновский объект "итерируемый" – означает, что  в цикле можно пройтись по каждому элементу этого объекта.

In [14]:
my_list = ['a', 'b', 'c', 'd', 'e']  # простой список
my_list

['a', 'b', 'c', 'd', 'e']

In [15]:
type(my_list)

list

Обход списка в цикле тремя способами:

In [16]:
# C/C++ style
for i in range(len(my_list)):
    print(my_list[i])

a
b
c
d
e


In [17]:
# python style
for element in my_list:
    print(element)

a
b
c
d
e


In [19]:
# combined style
for i, element in enumerate(my_list):
    print(i, element)

0 a
1 b
2 c
3 d
4 e


Аналогично работает для кортежей (tuple), словарей, множеств, строк и т.д.

Но не работает например для чисел:

In [21]:
a = 5
for number in a:
    print(number)

TypeError: 'int' object is not iterable

**Вопрос**: почему именно такое поведение и как это реализовано в Python?

**Ответ**: итерируемые классы поддерживают так называемый *протокол итераций* – *iterator protocol* (об этом ниже).

---
---

### 1. Выражения генераторов списков (list comprehension expression)

Способы создания списков:
*  вручную

In [26]:
my_list = [1, 2, 3, 4, 5]
my_list

[1, 2, 3, 4, 5]

*  поэлементным добавлением в цикле или функции

In [25]:
my_list = []
for i in range(1, 6):
    my_list.append(i)
my_list

[1, 2, 3, 4, 5]

*  с помощью выражения генераторов списков

In [27]:
my_list = [i for i in range(1, 6)]
my_list

[1, 2, 3, 4, 5]

Можно считать, что две предыдущие записи эквивалентны.

 ---

В чем преимущества создания списков с помощью "List Comprehension":
1. Читаемость кода
2. Скорость выполнения
3. Возможность использовать более сложные выражения (с условиями и итерированием по нескольким спискам)

In [29]:
def create_list_with_append(list_length=1000000):
    my_list = []
    for i in range(1, list_length+1):
        my_list.append(i)
    return my_list

In [30]:
def create_list_with_comprehension(list_length=1000000):
    return [i for i in range(1, list_length+1)]

In [34]:
%timeit create_list_with_append();

94.6 ms ± 862 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
%timeit create_list_with_comprehension();

58.2 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


 Примеры:

In [43]:
a = zip(['a', 'b', 'c'], [1, 2, 3])
a, list(a)

(<zip at 0x22e76a82b08>, [('a', 1), ('b', 2), ('c', 3)])

In [39]:
[k*v for (k, v) in zip(['a', 'b', 'c'], [1, 2, 3])]

['a', 'bb', 'ccc']

In [40]:
[x ** 2 if x % 2 == 0 else x ** 3 for x in range(10)]

[0, 1, 4, 27, 16, 125, 36, 343, 64, 729]

In [44]:
first = []
for x in range(1, 5):
    for y in range(5, 1, -1):
        if x != y:
            first.append((x, y))
            
# эквивалентно:
            
[(x, y) for x in range(1, 5) for y in range(5, 1, -1) if x != y]

[(1, 5),
 (1, 4),
 (1, 3),
 (1, 2),
 (2, 5),
 (2, 4),
 (2, 3),
 (3, 5),
 (3, 4),
 (3, 2),
 (4, 5),
 (4, 3),
 (4, 2)]

Но помните! — __Simple is better than complex__ (Простое лучше, чем сложное)*

*PEP 20 -- The Zen of Python  
_import this_ в интерпретаторе  
https://www.python.org/dev/peps/pep-0020/  
https://tyapk.ru/blog/post/the-zen-of-python

 ---

Приятный бонус – аналогичный синтаксис существует для создания __словарей__ и __множеств__:

In [167]:
my_dict = {key: value for key, value in zip([1, 2, 3], ['a', 'b', 'c'])}
type(my_dict), my_dict

(dict, {1: 'a', 2: 'b', 3: 'c'})

In [168]:
my_set = {key for key in [1, 1, 2, 2, 3, 3]}
type(my_set), my_set

(set, {1, 2, 3})

---
---

### 2. Выражения-генераторы (generator expressions)

__Выражения-генераторы__ – напоминают генераторы списков, но они не конструируют список с
результатами, а возвращают объект, который будет воспроизводить результаты по требованию.  
Поскольку такая конструкция не создает сразу весь список с результатами, она позволяют
__экономить память и производить дополнительные вычисления между операциями__ получения результатов.  
Такая возможность возврата результатов по требованию обеспечена за счёт реализации _протокола итераций_.

In [65]:
my_gen = (k*v for (k, v) in zip(['a', 'b', 'c', 'd', 'e'], [1, 2, 3, 4, 5]))
my_gen

<generator object <genexpr> at 0x0000022E76C3AA40>

In [66]:
# Получаем сразу все значения с помощью преобразования в список:
list(my_gen)

['a', 'bb', 'ccc', 'dddd', 'eeeee']

In [67]:
# Второй раз не получится, т.к. данный генератор уже "выдал" все свои значения!
list(my_gen)

[]

In [104]:
# Если хотим пройтись по значениям ещё раз, то придётся создавать генератор заного:
my_gen = (k*v for (k, v) in zip(['a', 'b', 'c', 'd', 'e'], [1, 2, 3, 4, 5]))
tuple(my_gen)

('a', 'bb', 'ccc', 'dddd', 'eeeee')

In [108]:
# Также значения можно получать в цикле:
my_gen = (i for i in range(3))
for i in my_gen:
    print(i)

0
1
2


In [100]:
# Или передавая генератор стандартной функции next():
my_gen = (i for i in range(3))
next(my_gen)

0

In [101]:
next(my_gen)

1

In [102]:
next(my_gen)

2

In [103]:
# Но тогда, при исчерпании значений, будет вызвано исключение StopIteration:
next(my_gen)

StopIteration: 

 ---

 ### ВАЖНО! Значения генерируются не в момент создания генератора, а в момент его вызова!

In [169]:
# Пример:

my_list = [1, 2, 3, 4, 5]
my_gen = (i*i for i in my_list)

# далее может быть любой код...
# в том числе и изменяющий исходный список:
my_list.append(6)

# теперь получаем итоговые значения:
list(my_gen)

[1, 4, 9, 16, 25, 36]

Выражение-генератор использовало обновленный список.

В отличие от генераторов-списков, которые рассчитывают итоговые значения СРАЗУ:

In [89]:
# Пример:

my_list = [1, 2, 3, 4, 5]
my_list_gen = [i*i for i in my_list]

# далее идёт какой-то код...
# в том числе и изменяющий исходный список:
my_list.append(6)

# получаем итоговые значения:
list(my_list_gen)

[1, 4, 9, 16, 25]

 ---

Остальные свойства генераторов-выражений аналогичны генераторам списков  
(можно использовать в заглавии цикла, с условными выражениями и с итерациями по нескольким значениям):

In [79]:
my_gen = ((x, y) for x in range(1, 5) for y in range(5, 1, -1) if x != y)
for x, y in my_gen:
    print(x, y)

1 5
1 4
1 3
1 2
2 5
2 4
2 3
3 5
3 4
3 2
4 5
4 3
4 2


---
---

### 3. Функции-генераторы (generator functions)

Что если нужна более сложная логика для генераторов?  
Для этого в Python существуют функции-генераторы – они выглядят как обычные инструкции для создания функций _def_, но для возврата результатов по одному значению за раз используют команду __yield__ (вместо __return__), приостанавливающую выполнение функции в данном месте программы до следующего вызова значения генератора.

In [131]:
# Пример обычной функции:
def get_random_numbers(N_numbers, a=1, b=100):
    from random import randint  # генерирует случайное целое число в диапазоне от 'a' до 'b'
    
    random_numbers_list = []
    
    for i in range(int(N_numbers)):
        random_numbers_list.append(randint(int(a), int(b)))
        
    return random_numbers_list  # здесь возвращается весь список

In [125]:
my_list = get_random_numbers(5)
my_list

[61, 20, 50, 90, 59]

In [157]:
# Аналогичная функция-генератор:
def get_random_numbers_generator(N_numbers, a=1, b=100):
    from random import randint  # генерирует случайное целое число в диапазоне от 'a' до 'b'
    
    for i in range(int(N_numbers)):
        random_number = randint(int(a), int(b))
        print('Получено число: {}'.format(random_number))
        
        yield random_number  # здесь происходит возврат одного значения и заморозка состояния до следующего раза
        print('Продолжаю генерацию...')
        
    print('Генерация случайных чисел окончена', end='\n\n')

In [158]:
random_numbers = get_random_numbers_generator(5)
random_numbers

<generator object get_random_numbers_generator at 0x0000022E7AFECFC0>

In [159]:
print(list(random_numbers))

Получено число: 74
Продолжаю генерацию...
Получено число: 79
Продолжаю генерацию...
Получено число: 76
Продолжаю генерацию...
Получено число: 65
Продолжаю генерацию...
Получено число: 97
Продолжаю генерацию...
Генерация случайных чисел окончена

[74, 79, 76, 65, 97]


In [161]:
# По прежнему генератор сработает только один раз (если он не бесконечный – но тогда (в данном коде) он бы завис):
print(list(random_numbers))

[]


In [172]:
# Само собой можно получать значения в цикле:
random_numbers = get_random_numbers_generator(5)

for number in random_numbers:
    print(number)

Получено число: 82
82
Продолжаю генерацию...
Получено число: 22
22
Продолжаю генерацию...
Получено число: 90
90
Продолжаю генерацию...
Получено число: 55
55
Продолжаю генерацию...
Получено число: 82
82
Продолжаю генерацию...
Генерация случайных чисел окончена



---
---

### 4. Протокол итераций (iterable protocol)

Что происходит, когда мы пишем:

    for element in my_generator:
        print(element)
        
1. В начале с помощью встроенного в генератор метода **\_\_iter\_\_**() вызывается объект-итератор.
2. Далее последовательно каждый раз возвращается генерируемое значение с помощью метод  **\_\_next_\_**().
3. В конце, при исчерпании генерируемых значений, вызвается исключение StopIteration, которое автоматически обрабатывается циклом.

__Это и есть протокол итерации (iterable protocol)__.

Таким образом, чтобы сделать любой класс итерируемым, достаточно реализовать в нем данные два метода.  
Пример:

In [99]:
class FibonacciBatchGenerator:
    """
    Бесконечный генератор последовательности чисел Фибоначчи
    
    
    Следующее число равно сумме двух предыдущих:    
        0, 1, 1, 2, 3, 5, 8, 13, 21...
    """
    
    def __init__(self, batch_size=10):

        self.batch_size = batch_size
        self.i = 0
        self.first = 1
        self.second = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):

        if self.i > 0:
            self.first, self.second = self.second, self.first+self.second
        
        if self.i < self.batch_size:
            self.i += 1
            return self.second
        else:
            self.i = 0  # возвращаем исходное значение, чтобы работал повторный вызова генератора
            raise StopIteration

In [100]:
fibonacci = FibonacciBatchGenerator(10)
list(fibonacci)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [101]:
# повторно работает, продолжает выдавать числа "порциями"
list(fibonacci)

[55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

In [102]:
# next работает для 10 чисел
print(
    next(fibonacci), next(fibonacci), next(fibonacci), next(fibonacci), next(fibonacci),
    next(fibonacci), next(fibonacci), next(fibonacci), next(fibonacci), next(fibonacci)
)

6765 10946 17711 28657 46368 75025 121393 196418 317811 514229


In [103]:
# а дальше функция next вызывет исключение StopIteration (как и обычный генератор)
next(fibonacci)

StopIteration: 

---
---
---

### 5. Исключения (Exceptions)

Исключения в языке Python возбуждаются автоматически, когда программный код допускает ошибку, а также могут возбуждаться и перехватываться самим программным кодом. Обрабатываются исключения следующими командами:

> **try / except** (**/ else / finally**) – перехватывает исключения, возбужденные интерпретатором или вашим программным
кодом, и выполняет восстановительные операции.

>**raise** – дает возможность возбудить исключение программно.

>**assert** – дает возможность возбудить исключение программно, при выполнении определенного условия.

>**with/as** – реализует менеджеры контекста.

Исключения позволяют перепрыгнуть через фрагмент программы произвольной длины и соответствующим образом реагировать на необычные события.

#### Built-in Exceptions

In [111]:
(Exception,
 OverflowError, ZeroDivisionError, FloatingPointError, RecursionError,
 ImportError, ModuleNotFoundError,
 OSError,
 IndexError, KeyError, StopIteration,
 NotImplementedError, IndentationError)

(Exception,
 OverflowError,
 ZeroDivisionError,
 FloatingPointError,
 RecursionError,
 ImportError,
 ModuleNotFoundError,
 OSError,
 IndexError,
 KeyError,
 StopIteration,
 NotImplementedError,
 IndentationError)

https://docs.python.org/3/library/exceptions.html

---

Варианты синтаксиса **try / except / else / finally**:

In [123]:
a = 10; b = 2

try:
    # Обязательный блок выполнения команд:
    division = a // b

except:
    # Обязательный блок обработки возникшего исключения:
    print('Невозможно поделить на ноль')

else:
    # Необязательный блок, но может существовать только при наличии блока 'except'.
    # Исполняется, если в блоке 'try' не возникло никаких исключений (т.е. он успешно выполнился):
    print('Результат деления: {} : {} = {}'.format(a, b, division))

finally:
    # Необязательный блок.
    # Исполняется ВСЕГДА, независимо от того, возникло ли исключение или нет.
    print('Выполнение финального блока закончено.')

Результат деления: 10 : 2 = 5
Выполнение финального блока закончено.


In [138]:
a = 10; b = 0

try:
    # Обязательный блок выполнения команд
    division = a // b

except ZeroDivisionError:
    # Необязательный блок обработки возникшего исключения деления на 0:
    print('Невозможно поделить на ноль.')

except Exception as E:
    # Необязательный блок обработки всех остальных возникших исключений:
    print("'{}' - '{}'".format(type(E), E))

finally:
    # Необязательный блок.
    # Исполняется ВСЕГДА, независимо от того, возникло ли исключение или нет.
    print('Выполнение финального блока закончено.')

Невозможно поделить на ноль.
Выполнение финального блока закончено.


In [139]:
a = 10; b = 'Z'

try:
    # Обязательный блок выполнения команд
    division = a // b

except ZeroDivisionError:
    # Необязательный блок обработки возникшего исключения деления на 0:
    print('Невозможно поделить на ноль.')

except Exception as E:
    # Необязательный блок обработки всех остальных возникших исключений:
    print("'{}' - '{}'".format(type(E), E))

'<class 'TypeError'>' - 'unsupported operand type(s) for //: 'int' and 'str''


In [140]:
a = 10; b = 'Z'

try:
    # Обязательный блок выполнения команд
    division = a // b

except (ZeroDivisionError, TypeError):
    # Необязательный блок обработки исключений двух типов:
    print('Невозможно поделить: {} и {}'.format(a, b))

Невозможно поделить: 10 и Z


In [141]:
a = 10; b = 'Z'

try:
    # Обязательный блок выполнения команд
    division = a // b

except:
    # Необязательный блок обработки исключений:
    try:
        division = b // a
        
    except:
        division = None
        
print('Division:', division)

Division: None


In [144]:
# except не обязателен
try:
    print('try')
finally:
    print('finally')

try
finally


In [145]:
# но одиночный блок 'try' НЕВОЗМОЖЕН
try:
    print('try')

SyntaxError: unexpected EOF while parsing (<ipython-input-145-d660c1d5373b>, line 3)

---

Варианты синтаксиса **raise**:

In [152]:
a = 10; b = 0

try:
    division = a // b
    
except ZeroDivisionError:
    print('Не могу поделить на ноль')
    raise ZeroDivisionError

Не могу поделить на ноль


ZeroDivisionError: 

In [153]:
a = 10; b = 'Z'

if type(a) != type(b):
    raise Exception('Разные типы данных')

Exception: Разные типы данных

In [157]:
class MyClass():
    
    def __init__(self):
        pass
    
    def get_size(self):
        raise NotImplementedError

In [158]:
my_class_instance = MyClass()
my_class_instance.get_size()

NotImplementedError: 

---

**Assert** – это условная форма инструкции raise, которая используется в основном для отладки в процессе
разработки, а также в процессе написания тестов.

Варианты синтаксиса **assert**:

In [159]:
a = 10; b = 0

assert b != 0

AssertionError: 

---

Можно создавать **свои классы исключений**, главное наследоваться от базового класса или его потомка.

In [ ]:
class ValidationError(Exception):
    def __init__(self, message, errors):

        # Call the base class constructor with the parameters it needs
        super().__init__(message)

        # Now for your custom code...
        self.errors = errors

---
---
---

А теперь вопрос: что вернёт эта функция при выполнении:

In [161]:
def some_func():
    try:
        return 'from_try'
    finally:
        return 'from_finally'

In [ ]:
some_func()

Взято отсюда (нестандартное и неожиданное поведение языка Python):
https://github.com/satwikkansal/wtfPython